In [31]:
from __future__ import print_function

import matplotlib.pyplot as plt
import os
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import linear_model, pipeline
#import category_encoders as ce

from sklearn import preprocessing
from collections import defaultdict

df = pd.read_csv("/Users/diniu/Desktop/mainTable.csv")
df.head()

,Unnamed: 0,odid,display_name,od_name,production_year,running_time,sequel,distributor,opening_weekend_revenue,opening_weekend_theaters,...,production_budget,domestic_box_office,international_box_office,inflation_adjusted_domestic_box_office,production_company,rating,castperson,billing,technical_person,role
0,0,10100,Titanic,Titanic,1997,194.0,0.0,Paramount Pictures,28638131,2674,...,200000000,658672302.0,1.548943e+09,1.147349e+09,20th Century Fox,PG-13,Leonardo DiCaprio,1.0,James Cameron,Director
1,1,10100,Titanic,Titanic,1997,194.0,0.0,Paramount Pictures,28638131,2674,...,200000000,658672302.0,1.548943e+09,1.147349e+09,Paramount Pictures,PG-13,Leonardo DiCaprio,1.0,James Cameron,Director
2,2,10100,Titanic,Titanic,1997,194.0,0.0,Paramount Pictures,28638131,2674,...,200000000,658672302.0,1.548943e+09,1.147349e+09,Lightstorm Entertainment,PG-13,Leonardo DiCaprio,1.0,James Cameron,Director
3,3,20100,The Dark Knight,Dark Knight The,2008,152.0,1.0,Warner Bros.,158411483,4366,...,185000000,533345358.0,4.695460e+08,6.260806e+08,Legendary Pictures,PG-13,Christian Bale,1.0,Christopher Nolan,Director
4,4,20100,The Dark Knight,Dark Knight The,2008,152.0,1.0,Warner Bros.,158411483,4366,...,185000000,533345358.0,4.695460e+08,6.260806e+08,Warner Bros.,PG-13,Christian Bale,1.0,Christopher Nolan,Director


In [32]:
clean_data = df[df['display_name'].notnull() & df['od_name'].notnull() & df['running_time'].notnull()
               & df['castperson'].notnull() & df['inflation_adjusted_domestic_box_office'].notnull() 
                & df['opening_weekend_revenue'].notnull() & df['opening_weekend_theaters'].notnull()
               & df['production_year'].notnull() & df['production_budget'].notnull() & df['genre'].notnull()]

In [33]:
x_list_pre = ['display_name','od_name','running_time', 'castperson', 'inflation_adjusted_domestic_box_office',
              'opening_weekend_revenue', 'opening_weekend_theaters', 'production_year','production_budget','genre']

In [40]:
x_list_enc = ['display_name','od_name', 'castperson']
x_list_add = ['running_time','production_year','production_budget','inflation_adjusted_domestic_box_office','genre']
x_list_train = ['display_name','od_name', 'castperson','running_time','production_year',
                'production_budget','inflation_adjusted_domestic_box_office','genre']

In [41]:
clean_data_slice = clean_data.ix[:,x_list_pre]
min_max=preprocessing.RobustScaler()

e_data_minmax=min_max.fit_transform(clean_data_slice[['inflation_adjusted_domestic_box_office', 'production_budget']])
clean_data_slice['production_budget'] = pd.DataFrame(e_data_minmax[:,0],index=clean_data_slice.index)
clean_data_slice['inflation_adjusted_domestic_box_office'] = pd.DataFrame(e_data_minmax[:,1],index=clean_data_slice.index)
center_data_slice = clean_data_slice[clean_data_slice['production_budget']<5]
center_data_slice.head(20)

df.loc[df['production_year'].idxmin()]


Unnamed: 0                                                 13327
odid                                                   175180100
display_name                              Le Voyage dans la Lune
od_name                                   Voyage dans la Lune Le
production_year                                             1902
running_time                                                  14
sequel                                                         0
distributor                                        Flicker Alley
opening_weekend_revenue                                        0
opening_weekend_theaters                                       0
maximum_theaters                                               2
theatrical_engagements                                         4
domestic_dvd_units                                           NaN
domestic_dvd_spending                                        NaN
domestic_bluray_units                                        NaN
domestic_bluray_spending 

In [42]:
le = defaultdict(preprocessing.LabelEncoder) 
s = center_data_slice['genre'].str.split('|').apply(pd.Series, 1)
del center_data_slice['genre']
s = s.fillna('')

genres_num = s.apply(lambda x: le[x.name].fit_transform(x))
genres_num.head()

,0
642,1
643,1
660,3
661,3
672,12


In [43]:
encode_slice=center_data_slice[x_list_enc]
encoded_data = encode_slice.apply(lambda x: le[x.name].fit_transform(x))
encoded_data = encoded_data.join(genres_num)
encoded_data.head()

,display_name,od_name,castperson,0
642,1404,1576,2878,1
643,1404,1576,2878,1
660,592,593,1346,3
661,592,593,1346,3
672,1048,1157,2495,12


In [44]:
encoded_data = encoded_data.join(clean_data_slice[x_list_add])
y_data = center_data_slice['inflation_adjusted_domestic_box_office']
encoded_data.head()

,display_name,od_name,castperson,0,running_time,production_year,production_budget,inflation_adjusted_domestic_box_office,genre
642,1404,1576,2878,1,87.0,2009,4.789462,6.521739,Adventure
643,1404,1576,2878,1,87.0,2009,4.789462,6.521739,Adventure
660,592,593,1346,3,96.0,2008,4.769983,6.521739,Comedy
661,592,593,1346,3,96.0,2008,4.769983,6.521739,Comedy
672,1048,1157,2495,12,111.0,2009,4.637026,1.304348,Thriller/Suspense


In [45]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(encoded_data, y_data, 
                                                    test_size=0.25, random_state=0)